In [1]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache, SQLiteCache
from langchain_core.output_parsers import StrOutputParser
from langchain_core.vectorstores import InMemoryVectorStore


from langchain_community.document_loaders import PyMuPDFLoader
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall

from datasets import Dataset
from ragas.llms import LangchainLLMWrapper
import pandas as pd

import os
import re

from dotenv import load_dotenv
load_dotenv()

COLLECTION_NAME = "tax_law"
PERSIST_DIRECTORY = "tax"

In [2]:
def get_pdf_file_names(folder_path):
    """
    주어진 폴더 내에 있는 PDF 파일들의 이름을 리스트로 반환합니다.

    Args:
        folder_path (str): PDF 파일이 있는 폴더의 경로.

    Returns:
        list: PDF 파일 이름들의 리스트.
    """
    try:
        # 폴더 내 파일 리스트 가져오기
        all_files = os.listdir(folder_path)
        
        # 확장자가 '.pdf'인 파일만 필터링
        pdf_files = [file.replace(".pdf","") for file in all_files if file.lower().endswith('.pdf')]
        
        return pdf_files
    except FileNotFoundError:
        print(f"Error: 폴더 '{folder_path}'를 찾을 수 없습니다.")
        return []
    except Exception as e:
        print(f"Error: {e}")
        return []

In [3]:
def load_and_split_tax_law(file_path,repeat_pattern):
    """
    PDF 파일을 로드하고, 반복 텍스트 제거, 조항 기준으로 분할 및 연결 후 임베딩 준비.
    Args:
        file_path (str): PDF 파일 경로.
    Returns:
        list: 연결된 텍스트 조각들의 리스트.
    """
    import re
    from langchain.document_loaders import PyMuPDFLoader
    from langchain_openai import OpenAIEmbeddings

    # PDF 로드
    loader = PyMuPDFLoader(file_path)
    documents = loader.load()
    
    # 전체 텍스트 가져오기
    full_text = " ".join([doc.page_content for doc in documents])

    # 1. 반복 텍스트 제거
    full_text = re.sub(repeat_pattern, "", full_text)

    # 2. '제n조'를 기준으로 분할
    split_pattern = r"\n(제\d+조[^\s]*)"
    chunks = re.split(split_pattern, full_text)

    # 3. '제n조'와 연결하여 완전한 문단 구성 + 부칙 처리
    connected_chunks = []
    current_chunk = ""
    is_buchik_section = False  # 부칙 여부 플래그

    for chunk in chunks:
        if re.search(r"\n\s*<부칙>", chunk):  # 부칙 시작 여부 확인
            is_buchik_section = True

        if chunk.startswith("제") and "조" in chunk:  # 새로운 조항 시작
            if is_buchik_section:  # 부칙 섹션이라면 접두사 추가
                chunk = "부칙-" + chunk

            if current_chunk:  # 이전 조항 저장
                connected_chunks.append(current_chunk.strip())
            current_chunk = chunk  # 새 조항으로 시작
        else:
            current_chunk += f" {chunk}"  # 기존 조항에 내용 추가

    if current_chunk:
        connected_chunks.append(current_chunk.strip())


    return connected_chunks, full_text



In [4]:
def get_vector_store(pdf_files):
    """
    PDF 파일들을 처리하여 임베딩을 Chroma Vector Store에 저장합니다.

    Args:
        pdf_files (list): PDF 파일 이름들의 리스트.
        vector_store_path (str): Chroma Vector Store를 저장할 경로.
    """
    from langchain_chroma import Chroma
    from langchain_core.documents import Document
    
    # 각 PDF 파일에 대해 임베딩 처리
    all_docs = []
    
    for file in pdf_files:
        path = f"data/tax_law/{file}.pdf"  # PDF 파일 경로
        repeat_pattern = rf"법제처\s*\d+\s*국가법령정보센터\n{file.replace('_', ' ')}\n" 
        chunks, full_text = load_and_split_tax_law(path, repeat_pattern)
        

        for idx, chunk in enumerate(chunks):
            _doc = Document(metadata={"title": file, "full_text": full_text[idx], }, page_content=chunk),
            all_docs.extend(_doc)

    embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")
    
    # 📌 중복 될 수 있어서 주석 처리함!
    # vector_store = Chroma.from_documents(
    #     documents=all_docs,
    #     embedding=embedding_model,
    #     collection_name=COLLECTION_NAME,
    #     persist_directory=PERSIST_DIRECTORY
    # )
    
    # vector_store =  Chroma(
    #     collection_name=COLLECTION_NAME,
    #     persist_directory=PERSIST_DIRECTORY,
    #     embedding_function=embedding_model,
    # )
        
    return vector_store
        
folder_path = "data/tax_law" 
pdf_files = get_pdf_file_names(folder_path)
vector_store = get_vector_store(pdf_files)

c:\Users\Playdata\AppData\Local\miniconda3\envs\langchain\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:300: UserWarning: Warning: Empty content on page 0 of document data/tax_law/2024_new_tax.pdf
  warnings.warn(
c:\Users\Playdata\AppData\Local\miniconda3\envs\langchain\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:300: UserWarning: Warning: Empty content on page 1 of document data/tax_law/2024_new_tax.pdf
  warnings.warn(
c:\Users\Playdata\AppData\Local\miniconda3\envs\langchain\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:300: UserWarning: Warning: Empty content on page 15 of document data/tax_law/2024_new_tax.pdf
  warnings.warn(
c:\Users\Playdata\AppData\Local\miniconda3\envs\langchain\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:300: UserWarning: Warning: Empty content on page 17 of document data/tax_law/2024_new_tax.pdf
  warnings.warn(
c:\Users\Playdata\AppData\Local\miniconda3\env

In [5]:
embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")

COLLECTION_NAME = "tax_law"
PERSIST_DIRECTORY = "tax"

vector_store =  Chroma(
    collection_name=COLLECTION_NAME,
    persist_directory=PERSIST_DIRECTORY,
    embedding_function=embedding_model,
)

In [6]:
vector_store._collection.count()

5734

In [7]:
retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={"k":5, "fetch_k":10}
)


In [48]:
# Prompt Template 생성
messages = [
        ("ai", """
        당신은 대한민국 세법에 대해 전문적으로 학습된 AI 도우미입니다. 저장된 세법 조항 데이터를 기반으로 사용자 질문에 답변하세요.

        - 모든 답변은 학습된 세법 데이터 내에서만 유효한 정보를 바탕으로 작성하세요. 데이터에 없는 내용은 추측하거나 임의로 생성하지 마세요.
        - 질문에 명확한 답변이 없거나 데이터 내에서 찾을 수 없는 경우, 정직하게 "잘 모르겠습니다."라고 말하고, 새로운 질문을 유도하세요.
        - 질문이 포함된 조항뿐 아니라, 필요 시 서로 연관된 다른 조항도 참고하여 답변의 정확성과 완성도를 높이세요.
        - 사용자가 이해하기 쉽게 답변을 구성하며, 중요한 키워드나 법 조항은 명확히 표시하세요.
        - 세법과 관련된 복잡한 질문에 대해서는 관련 조항 번호와 요약된 내용을 포함하여 답변을 제공하세요.
        
        추가 규칙:
        답변은 간결하고 명료하게 작성하되, 필요한 경우 관련 조항의 전문을 추가적으로 인용하세요.
        세법 용어를 사용자 친화적으로 설명하여 비전문가도 쉽게 이해할 수 있도록 하세요.
        질문을 완전히 이해하기 어렵거나 모호할 경우, 사용자가 구체적으로 질문을 다시 작성할 수 있도록 유도하는 후속 질문을 하세요.

	답변 후, 사용자에게 필요할 것 같은 정보를 바탕으로 두 가지 후속 질문을 제안하세요. 각 질문의 앞뒤에 한 줄씩 띄어쓰기를 하세요. 이 질문은 원래 주제와 관련된 내용이어야 합니다.
	특정 법률 조항이나 제도가 언급될 경우, 근거가 되는 세법 조문, 시행령, 또는 관련 자료를 명시합니다.
        모든 답변은 사용자에게 법적 조언이 아닌 정보 제공 목적으로 작성된 것임을 명확히 합니다. 
	{context}")"""
        ),
        ("human", "{question}"),
]
prompt_template = ChatPromptTemplate(messages)
# 모델
model = ChatOpenAI(model="gpt-4o")

# output parser
parser = StrOutputParser()

# Chain 구성 retriever(관련문서 조회) -> prompt_template(prompt 생성) -> model(정답) -> output parser
chain = {"context":retriever, "question": RunnablePassthrough()} | prompt_template | model | parser


## 평가

In [ ]:
# LangChain 모델 래핑
langchain_model = LangchainLLMWrapper(model)

# 테스트 데이터 준비 (예시)
test_data = [
    {
        "question": "개별소비세법의 목적은 무엇인가요?",
        "answer": chain.invoke("개별소비세법의 목적은 무엇인가요?"),
        "contexts": [doc.page_content for doc in retriever.get_relevant_documents("개별소비세법의 목적은 무엇인가요?")],
        "ground_truths": ["개별소비세법의 목적은 특정 물품과 특정 장소에 대한 소비세를 부과하여 국가 재정수입을 확보하고 소비를 조정하는 것입니다."],
        "reference": "\n".join([doc.page_content for doc in retriever.get_relevant_documents("개별소비세법의 목적은 무엇인가요?")])
    },
    # {
    #     "question": "개별소비세법 제1조가 무엇인가요?",
    #     "answer": chain.invoke("개별소비세법 제1조가 무엇인가요?"),
    #     "contexts": [doc.page_content for doc in retriever.get_relevant_documents("개별소비세법 제1조가 무엇인가요?")],
    #     "ground_truths": ["개별소비세는 특정한 물품, 특정한 장소 입장행위(入場行爲), 특정한 장소에서의 유흥음식행위(遊興飮食行爲) 및 특정한 장소에서의 영업행위에 대하여 부과한다."],
    #     "reference": "\n".join([doc.page_content for doc in retriever.get_relevant_documents("개별소비세법 제1조가 무엇인가요?")])
    # },
    {
        "question": "조세범 처벌절차법의 목적이 무엇인가요?",
        "answer": chain.invoke("조세범 처벌절차법의 목적이 무엇인가요?"),
        "contexts": [doc.page_content for doc in retriever.get_relevant_documents("조세범 처벌절차법의 목적이 무엇인가요?")],
        "ground_truths": ["제1조(목적) 이 법은 조세범칙사건(犯則事件)을 공정하고 효율적으로 처리하기 위하여 조세범칙사건의 조사 및 그 처분에 관한 사항을 정함을 목적으로 한다"],
        "reference": "\n".join([doc.page_content for doc in retriever.get_relevant_documents("조세범 처벌절차법의 목적이 무엇인가요?")])
    },
    
]

# Dataset 생성
dataset = Dataset.from_list(test_data)

# 평가 실행
result = evaluate(
    dataset,
    metrics=[
        faithfulness,
        answer_relevancy,
        context_precision,
        context_recall,
    ],
    llm=langchain_model,
)

# 결과 출력
print(result)


Evaluating:   0%|          | 0/8 [00:00<?, ?it/s]

{'faithfulness': 0.3000, 'answer_relevancy': 0.8789, 'context_precision': 1.0000, 'context_recall': 1.0000}


In [51]:
# DataFrame 생성
df_result = pd.DataFrame(test_data)

# 최대 열 너비 설정
pd.set_option('display.max_colwidth', None)

# 최대 행 수 설정
pd.set_option('display.max_rows', None)

df_result

,question,answer,contexts,ground_truths,reference
0,개별소비세법의 목적은 무엇인가요?,"개별소비세법의 목적은 특정 물품이나 서비스에 대해 소비세를 부과하여 세수 확보와 소비 패턴 조정을 통해 공공재원을 조달하고 사회적 비용을 반영하는 것입니다. 이를 통해 사치품이나 환경에 미치는 영향이 큰 제품의 소비를 억제하고, 공공 목적으로 사용되는 자금을 확보하는 데 기여합니다. \n\n추가적으로 궁금하신 사항이 있나요?\n\n1. 개별소비세가 부과되는 구체적인 품목에는 어떤 것들이 있는지 알고 싶으신가요?\n \n2. 개별소비세의 세율은 어떻게 결정되는지 궁금하신가요?","[개별소비세법 시행규칙\n[시행 2024. 3. 22.] [기획재정부령 제1047호, 2024. 3. 22., 일부개정]\n기획재정부 (환경에너지세제과) 044-215-4331, 4336\n기획재정부 (환경에너지세제과 - 자동차 부분) 044-215-4333, 4336, 제4조(과세시기) 개별소비세는 다음 각 호에 따른 반출, 수입신고, 입장, 유흥음식행위 또는 영업행위를 할 때에 그 행위\n당시의 법령에 따라 부과한다. 다만, 제3조제4호의 경우에는 「관세법」에 따른다. <개정 2015. 12. 15.>\n1. 물품에 대한 개별소비세: 과세물품을 제조장에서 반출할 때 또는 수입신고를 할 때\n2. 입장행위에 대한 개별소비세: 과세장소에 입장할 때\n3. 유흥음식행위에 대한 개별소비세: 유흥음식행위를 할 때\n 4. 영업행위에 대한 개별소비세: 과세영업장소의 영업행위를 할 때\n[전문개정 2010. 1. 1.], 제1조(목적) 이 법은 주세의 과세 요건 및 절차를 규정함으로써 주세를 공정하게 과세하고, 납세의무의 적정한 이행을\n확보하며, 재정수입의 원활한 조달에 이바지함을 목적으로 한다., 제1조(목적) 이 규칙은 「외국인관광객 등에 대한 부가가치세 및 개별소비세 특례규정」에서 위임된 사항과 그 시행에\n필요한 사항을 규정함을 목적으로 한다.\n[전문개정 2010. 4. 13.], 제1조(목적) 이 법은 조세(租稅)의 감면 또는 중과(重課) 등 조세특례와 이의 제한에 관한 사항을 규정하여 과세(課稅)의\n공평을 도모하고 조세정책을 효율적으로 수행함으로써 국민경제의 건전한 발전에 이바지함을 목적으로 한다. <개정\n2020. 6. 9.>\n[전문개정 2010. 1. 1.]]",[개별소비세법의 목적은 특정 물품과 특정 장소에 대한 소비세를 부과하여 국가 재정수입을 확보하고 소비를 조정하는 것입니다.],"개별소비세법 시행규칙\n[시행 2024. 3. 22.] [기획재정부령 제1047호, 2024. 3. 22., 일부개정]\n기획재정부 (환경에너지세제과) 044-215-4331, 4336\n기획재정부 (환경에너지세제과 - 자동차 부분) 044-215-4333, 4336\n제4조(과세시기) 개별소비세는 다음 각 호에 따른 반출, 수입신고, 입장, 유흥음식행위 또는 영업행위를 할 때에 그 행위\n당시의 법령에 따라 부과한다. 다만, 제3조제4호의 경우에는 「관세법」에 따른다. <개정 2015. 12. 15.>\n1. 물품에 대한 개별소비세: 과세물품을 제조장에서 반출할 때 또는 수입신고를 할 때\n2. 입장행위에 대한 개별소비세: 과세장소에 입장할 때\n3. 유흥음식행위에 대한 개별소비세: 유흥음식행위를 할 때\n 4. 영업행위에 대한 개별소비세: 과세영업장소의 영업행위를 할 때\n[전문개정 2010. 1. 1.]\n제1조(목적) 이 법은 주세의 과세 요건 및 절차를 규정함으로써 주세를 공정하게 과세하고, 납세의무의 적정한 이행을\n확보하며, 재정수입의 원활한 조달에 이바지함을 목적으로 한다.\n제1조(목적) 이 규칙은 「외국인관광객 등에 대한 부가가치세 및 개별소비세 특례규정」에서 위임된 사항과 그 시행에\n필요한 사항을 규정함을 목적으로 한다.\n[전문개정 2010. 4. 13.]\n제1조(목적) 이 법은 조세(租稅)의 감면 또는 중과(重課) 등 조세특례와 이의 제한에 관한 사항을 규정하여 과세(課稅)의\n공평을 도모하고 조세정책을 효율적으로 수행함으로써 국민경제의 건전한 발전에 이바지함을 목적으로 한다. <개정\n2020. 6. 9.>\n[전문개정 2010. 1. 1.]"
1,조세범 처벌절차법의 목적이 무엇인가요?,조세범 처벌절차법의 목적은 세법을 위반한 자에 대한 형벌에 관한 사항을 규정하여 세법의 실효성을 높이고 국민의 건전한 납세 의식을 확립하는 데 있습니다. 이 법은 세법 집행의 공정성과 세금 납부의 적정성을 보장하기 위한 제도적 장치를 마련하고 있습니다.\n\n세법과 관련된 다른 질문이 있으신가요? 예를 들어:\n\n1. 조세범 처벌절차법에 의해 처벌받는 구체적인 행위는 어떤 것이 있는지 알고 싶으신가요?\n\n2. 조세범 처벌절차법과 관련된 최신 개정 사항이 궁금하신가요?,"[조세범 처벌절차법\n[시행 2023. 1. 17.] [법률 제19212호, 2023. 1. 17., 일부개정]\n기획재정부 (조세법령운용팀) 044-215-4151\n 제1장 총칙, 제1조(목적) 이 영은 「조세범 처벌절차법」에서 위임된 사항과 그 시행에 필요한 사항을 규정함을 목적으로 한다., 제1조(목적) 이 법은 주세의 과세 요건 및 절차를 규정함으로써 주세를 공정하게 과세하고, 납세의무의 적정한 이행을\n확보하며, 재정수입의 원활한 조달에 이바지함을 목적으로 한다., 제1조(목적) 이 법은 세법을 위반한 자에 대한 형벌에 관한 사항을 규정하여 세법의 실효성을 높이고 국민의 건전한 납\n세의식을 확립함을 목적으로 한다. <개정 2018. 12. 31.>, 제39조(벌칙) 재평가세에 관한 범칙행위에 대하여는 조세범처벌법을 적용한다.]",[제1조(목적) 이 법은 조세범칙사건(犯則事件)을 공정하고 효율적으로 처리하기 위하여 조세범칙사건의 조사 및 그 처분에 관한 사항을 정함을 목적으로 한다],"조세범 처벌절차법\n[시행 2023. 1. 17.] [법률 제19212호, 2023. 1. 17., 일부개정]\n기획재정부 (조세법령운용팀) 044-215-4151\n 제1장 총칙\n제1조(목적) 이 영은 「조세범 처벌절차법」에서 위임된 사항과 그 시행에 필요한 사항을 규정함을 목적으로 한다.\n제1조(목적) 이 법은 주세의 과세 요건 및 절차를 규정함으로써 주세를 공정하게 과세하고, 납세의무의 적정한 이행을\n확보하며, 재정수입의 원활한 조달에 이바지함을 목적으로 한다.\n제1조(목적) 이 법은 세법을 위반한 자에 대한 형벌에 관한 사항을 규정하여 세법의 실효성을 높이고 국민의 건전한 납\n세의식을 확립함을 목적으로 한다. <개정 2018. 12. 31.>\n제39조(벌칙) 재평가세에 관한 범칙행위에 대하여는 조세범처벌법을 적용한다."
